In [189]:
import pyqrcodeng as pyqrcode #!
import random
import numpy as np
import torch
import os
import cv2

from torch.utils.data import Dataset, DataLoader



In [190]:
class QRCode:
    ERROR_LEVELS = ['L', 'M', 'Q', 'H']
    VERSION_NUMBERS = list(range(1, 41))
    MODES = 'binary', 'numeric', 'alphanumeric'

    def choise_version(self):
        return random.choice(self.VERSION_NUMBERS)
    
    def choise_level(self):
        return random.choice(self.ERROR_LEVELS)
    
    def choise_mode(self):
        return random.choice(self.MODES)
    
    def generate_data(self):
        mode_id = pyqrcode.tables.modes[self.mode]
        capacity = pyqrcode.tables.data_capacity[self.version][self.error_level][mode_id]
        
        match self.mode:
            case 'alphanumeric':
                return "".join(map(chr, np.random.choice(list(pyqrcode.tables.ALPHANUMERIC_CHARS), capacity)))
            case 'numeric':
                return "".join(map(str, np.random.choice(list(range(10)), capacity)))
            case 'binary':
                return np.random.bytes(capacity)
            case _:
                assert False and f"unexpected mode {self.mode}"

    def __init__(self, version: int = None, error_level = None, mode = None):
        self.version = version or self.choise_version()
        self.error_level = error_level or self.choise_level()
        self.mode = mode or self.choise_mode()
        
        self.code = pyqrcode.create(
            self.generate_data(), 
            error=self.error_level, 
            version=self.version, 
            mode=self.mode
        )
        
    def save_svg(self, path):
        self.code.svg(path, scale=1)
    
    def save_png(self, path):
        self.code.png(path, scale=1)
    
    def show(self):
        self.code.show()

In [187]:
DATASET_DIRECTORY = "./qr-code-dataset/"
os.mkdir(DATASET_DIRECTORY)

DATASET_SIZE = 10
for i in range(DATASET_SIZE):
    code = QRCode()
    code.save_svg(f"{DATASET_DIRECTORY}/{i}.svg")
    code.save_png(f"{DATASET_DIRECTORY}/{i}.png")
    

In [175]:

def centroids(code: QRCode):
    scale = 10
    centroid_size = 2
    offset = (scale - centroid_size) // 2
    centroids = Svg()
    oh = ShapeBuilder()
    for y, line in enumerate(code.text().split()):
        for x, char in enumerate(line):
            if char == '0':
                continue
            centroids.addElement(oh.createRect(x * scale + offset, y * scale + offset, centroid_size, centroid_size, strokewidth=0, fill='red'))
    return centroids
                

In [91]:
code = QRCode()

code.svg("code.svg")
centroids(code).save('centroids.svg')

AttributeError: 'QRCode' object has no attribute 'svg'